In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install torch torchvision torchmetrics

import argparse
import builtins
import math
import os
import random
import shutil
import time
import warnings
from datetime import datetime

import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import torchmetrics
import torch.optim

import numpy as np
from torch.utils.data import Dataset

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 13.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nv

In [2]:
class ImageEncoder(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = models.resnet50(weights=None)
        self.net.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.net.maxpool = nn.Identity()
        #self.net.fc = nn.Identity()  # You might still declare it to avoid errors, but it won't be used.
        del self.net.fc

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Manually handle each step, avoiding the original 'fc' layer
        x = self.net.conv1(x)
        x = self.net.bn1(x)
        x = self.net.relu(x)
        x = self.net.maxpool(x)  # Identity, so no effect
        x = self.net.layer1(x)
        x = self.net.layer2(x)
        x = self.net.layer3(x)
        x = self.net.layer4(x)
        x = self.net.avgpool(x)  # AdaptiveAvgPool2d at the end of ResNet
        x = torch.flatten(x, 1)
        # Do not call self.net.fc(x) as 'fc' is removed
        return x

class MoCo(nn.Module):
    """
    Build a MoCo model with: a query encoder, a key encoder, and a queue
    https://arxiv.org/abs/1911.05722
    """

    def __init__(self, base_encoder, dim=128, K=65536, m=0.999, T=0.07, size=224, mlp=False):
        """
        dim: feature dimension (default: 128)
        K: queue size; number of negative keys (default: 65536)
        m: moco momentum of updating key encoder (default: 0.999)
        T: softmax temperature (default: 0.07)
        """
        super(MoCo, self).__init__()

        self.K = K
        self.m = m
        self.T = T
        self.size = size

        # create the encoders
        # num_classes is the output fc dimension
        self.encoder_q = base_encoder()
        self.encoder_k = base_encoder()

        if mlp:  # hack: brute-force replacement
            dim_mlp = self.encoder_q(torch.zeros(1, 1, self.size, self.size)).shape[1]
            self.encoder_q = nn.Sequential(
                self.encoder_q,
                nn.Linear(dim, dim), nn.ReLU(), nn.Linear(dim, dim)
            )
            self.encoder_k = nn.Sequential(
                self.encoder_k,
                nn.Linear(dim, dim), nn.ReLU(), nn.Linear(dim, dim)
            )

        for param_q, param_k in zip(
            self.encoder_q.parameters(), self.encoder_k.parameters()
        ):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient

        # create the queue
        self.register_buffer("queue", torch.randn(dim, K))
        self.queue = torch.randn(dim, K)
        self.queue = nn.functional.normalize(self.queue, dim=0)

        self.register_buffer("queue_ptr", torch.zeros(1, dtype=torch.long))

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update of the key encoder
        """
        for param_q, param_k in zip(
            self.encoder_q.parameters(), self.encoder_k.parameters()
        ):
            param_k.data = param_k.data * self.m + param_q.data * (1.0 - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        # gather keys before updating queue
        #keys = concat_all_gather(keys)

        batch_size = keys.shape[0]

        ptr = int(self.queue_ptr)
        assert self.K % batch_size == 0  # for simplicity

        # replace the keys at ptr (dequeue and enqueue)
        self.queue[:, ptr : ptr + batch_size] = keys.T
        ptr = (ptr + batch_size) % self.K  # move pointer

        self.queue_ptr[0] = ptr

    @torch.no_grad()
    def _batch_shuffle_ddp(self, x):
        """
        Batch shuffle, for making use of BatchNorm.
        *** Only support DistributedDataParallel (DDP) model. ***
        """
        # gather from all gpus
        batch_size_this = x.shape[0]
        x_gather = concat_all_gather(x)
        batch_size_all = x_gather.shape[0]

        num_gpus = batch_size_all // batch_size_this

        # random shuffle index
        idx_shuffle = torch.randperm(batch_size_all).to(device)

        # broadcast to all gpus
        torch.distributed.broadcast(idx_shuffle, src=0)

        # index for restoring
        idx_unshuffle = torch.argsort(idx_shuffle)

        # shuffled index for this gpu
        gpu_idx = torch.distributed.get_rank()
        idx_this = idx_shuffle.view(num_gpus, -1)[gpu_idx]

        return x_gather[idx_this], idx_unshuffle

    @torch.no_grad()
    def _batch_unshuffle_ddp(self, x, idx_unshuffle):
        """
        Undo batch shuffle.
        *** Only support DistributedDataParallel (DDP) model. ***
        """
        # gather from all gpus
        batch_size_this = x.shape[0]
        x_gather = concat_all_gather(x)
        batch_size_all = x_gather.shape[0]

        num_gpus = batch_size_all // batch_size_this

        # restored index for this gpu
        gpu_idx = torch.distributed.get_rank()
        idx_this = idx_unshuffle.view(num_gpus, -1)[gpu_idx]

        return x_gather[idx_this]

    def forward(self, im_q, im_k):
        """
        Input:
            im_q: a batch of query images
            im_k: a batch of key images
        Output:
            logits, targets
        """

        # compute query features
        q = self.encoder_q(im_q)  # queries: NxC
        q = nn.functional.normalize(q, dim=1)

        # compute key features
        with torch.no_grad():  # no gradient to keys
            self._momentum_update_key_encoder()  # update the key encoder

            k = self.encoder_k(im_k)  # keys: NxC
            k = nn.functional.normalize(k, dim=1)

        # compute logits
        # Einstein sum is more intuitive
        # positive logits: Nx1
        l_pos = torch.einsum("nc,nc->n", [q, k]).unsqueeze(-1)
        # negative logits: NxK
        l_neg = torch.einsum("nc,ck->nk", [q, self.queue.clone().detach()])

        # logits: Nx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators
        labels = torch.zeros(logits.shape[0], dtype=torch.long).to(logits.device)

        # dequeue and enqueue
        self._dequeue_and_enqueue(k)

        return logits, labels


# utils
@torch.no_grad()
def concat_all_gather(tensor):
    """
    Performs all_gather operation on the provided tensors.
    *** Warning ***: torch.distributed.all_gather has no gradient.
    """
    tensors_gather = [
        torch.ones_like(tensor) for _ in range(torch.distributed.get_world_size())
    ]
    torch.distributed.all_gather(tensors_gather, tensor, async_op=False)

    output = torch.cat(tensors_gather, dim=0)
    return output

In [3]:
import random

from PIL import ImageFilter


class TwoCropsTransform:
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform):
        self.base_transform = base_transform

    def __call__(self, x):
        q = self.base_transform(x)
        k = self.base_transform(x)
        return [q, k]


class GaussianBlur:
    """Gaussian blur augmentation in SimCLR https://arxiv.org/abs/2002.05709"""

    def __init__(self, sigma=[0.1, 2.0]):
        self.sigma = sigma

    def __call__(self, x):
        sigma = random.uniform(self.sigma[0], self.sigma[1])
        x = x.filter(ImageFilter.GaussianBlur(radius=sigma))
        return x

In [4]:
def main():
    start_time = datetime.now()
    # Define arguments directly in the code for simplicity
    class Args:
        data = '/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/SimCLR/ElectricMotorFaults_processed.npz'  # Update this as needed
        arch = "resnet50"
        workers = 1
        epochs = 200 #base:200
        start_epoch = 0
        size = 24 #size of the image in NxN pixels
        batch_size = 128  # Lowering for Colab
        lr = 0.03
        schedule = [120, 160]
        momentum = 0.9
        weight_decay = 1e-4
        print_freq = 10
        resume = ""  # If needed, specify checkpoint path
        seed = None
        gpu = 0
        distributed = False

        # MoCo-specific configs
        moco_dim = 2048
        moco_k = 65536
        moco_m = 0.99 #def = 0.999
        moco_t = 0.07
        mlp = False
        aug_plus = True
        cos = False

    args = Args()

    if args.seed is not None:
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        cudnn.deterministic = True

    # Check for GPU availability
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Create the model
    print("=> creating model '{}'".format(args.arch))

    model = MoCo(
        ImageEncoder,
        args.moco_dim,
        args.moco_k,
        args.moco_m,
        args.moco_t,
        args.size,
        args.mlp,
    )
    model = model.to(device)
    print(model)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss().to(device)

    optimizer = torch.optim.SGD(
        model.parameters(),
        args.lr,
        momentum=args.momentum,
        weight_decay=args.weight_decay,
    )

    # Optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume, map_location=device)
            args.start_epoch = checkpoint["epoch"]
            model.load_state_dict(checkpoint["state_dict"])
            optimizer.load_state_dict(checkpoint["optimizer"])
            print("=> loaded checkpoint '{}' (epoch {})".format(args.resume, checkpoint["epoch"]))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    traindir = os.path.join(args.data, "train")
    normalize = transforms.Normalize(
        mean=[0.485], std=[0.229]
    )

    if args.aug_plus:
        # MoCo v2's aug adjusted to greyscale
        augmentation = [
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(args.size, scale=(0.2, 1.0)),
            transforms.RandomApply([transforms.ColorJitter(0.2,0.2)], p=0.8),
            transforms.RandomApply([GaussianBlur([0.1, 2.0])], p=0.5),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]
    else:
        # MoCo v1's aug adjusted to greyscale
        augmentation = [
            transforms.ToPILImage(),
            transforms.RandomResizedCrop(args.size, scale=(0.2, 1.0)),
            transforms.ColorJitter(0.2, 0.2),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]

    two_crops_transform = TwoCropsTransform(transforms.Compose(augmentation))

    train_dataset = NPZDataset(args.data, mode='train', transform=two_crops_transform)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.workers,
        pin_memory=True,
        drop_last=True,
    )

    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer, epoch, args)
        train(train_loader, model, criterion, optimizer, epoch, args, device)

        # Save checkpoint every 10 epochs
        if epoch % 199 == 0:
            save_checkpoint(
                {
                    "epoch": epoch + 1,
                    "arch": args.arch,
                    "state_dict": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                },
                filename="/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/MoCo/checkpoint_{:04d}.pth.tar".format(epoch),
            )
    print(f"Training time is : {datetime.now()-start_time}")

def train(train_loader, model, criterion, optimizer, epoch, args, device):
    batch_time = AverageMeter("Time", ":6.3f")
    data_time = AverageMeter("Data", ":6.3f")
    losses = AverageMeter("Loss", ":.4e")
    top1 = AverageMeter("Acc@1", ":6.2f")
    top5 = AverageMeter("Acc@5", ":6.2f")
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch),
    )

    # Switch to train mode
    model.train()

    end = time.time()

    for i, (images, _) in enumerate(train_loader):
        data_time.update(time.time() - end)

        images[0] = images[0].to(device, non_blocking=True)
        images[1] = images[1].to(device, non_blocking=True)

        output, target = model(im_q=images[0], im_k=images[1])
        loss = criterion(output, target)

        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images[0].size(0))
        top1.update(acc1[0], images[0].size(0))
        top5.update(acc5[0], images[0].size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)

def save_checkpoint(state, filename="/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/MoCo/checkpoint.pth.tar"):
    torch.save(state, filename)

class AverageMeter:
    def __init__(self, name, fmt=":f"):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = "{name} {val" + self.fmt + "} ({avg" + self.fmt + "})"
        return fmtstr.format(**self.__dict__)

class ProgressMeter:
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print("\t".join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches))
        fmt = "{:" + str(num_digits) + "d}"
        return "[" + fmt + "/" + fmt.format(num_batches) + "]"

def adjust_learning_rate(optimizer, epoch, args):
    lr = args.lr
    if args.cos:
        lr *= 0.5 * (1.0 + math.cos(math.pi * epoch / args.epochs))
    else:
        for milestone in args.schedule:
            lr *= 0.1 if epoch >= milestone else 1.0
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True) #view -> reshape
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [5]:
class NPZDataset(Dataset):
    def __init__(self, file_path, mode='train', transform=None):
        data = np.load(file_path)
        self.images = data[f'{mode}_images']
        self.labels = data[f'{mode}_labels']  # Note: Labels might not be used in MoCo training
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
          q, k = self.transform(image)
        return [q, k], idx  # MoCo typically doesn't use labels directly in the loss calculation

In [6]:
main()

=> creating model 'resnet50'
MoCo(
  (encoder_q): ImageEncoder(
    (net): ResNet(
      (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): Identity()
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (downsample): Seque

In [ ]:
from google.colab import runtime
runtime.unassign()

In [7]:
#!/usr/bin/env python
# Copyright (c) Meta Platforms, Inc. and affiliates.

# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

class LinClrDataset(Dataset):
    def __init__(self, file_path, mode='train', transform=None):
        data = np.load(file_path)
        self.images = data[f'{mode}_images']
        self.labels = data[f'{mode}_labels']  # Note: Labels might not be used in MoCo training
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            q = self.transform(image)
        labels = self.labels[idx]
        return q, labels


class CustomImageClassifier(nn.Module):
    def __init__(self, num_classes, dim):
        super().__init__()
        self.encoder = ImageEncoder()
        self.dim = dim
        # Assume the output features of ImageEncoder are 2048-dimensional
        self.fc = nn.Linear(self.dim, num_classes)

    def forward(self, x):
        features = self.encoder(x)
        output = self.fc(features)
        return output


model_names = sorted(
    name
    for name in models.__dict__
    if name.islower() and not name.startswith("__") and callable(models.__dict__[name])
)

class Args:
    data = '/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/SimCLR/ElectricMotorFaults_processed.npz'  # path to dataset
    arch = "resnet50" #model architecture, default: resnet50
    workers = 1 #number of loading workers, def:32
    epochs = 100 #total numnber of epochs, def: 100
    start_epoch = 0 #(for restarts)
    batch_size = 128 #mini-batch size
    lr = 30.0 #initial learning rate
    schedule = [60, 80] #learning rate schedule, when to drop lr by a ratio
    momentum = 0.9
    weight_decay = 0.0
    print_freq = 10
    resume = "" #path to latest checkpoint
    evaluate = False #evaluate model on validation set
    world_size = -1 #number of nodes for distributed training
    rank = -1 #node rank for distrivuted training
    dist_url = "tcp://224.66.41.62:23456" #url used for distributed training
    dist_backend = "nccl" #distributed backend
    seed = None
    gpu = 0
    multiprocessing_distributed = False
    pretrained = "/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/MoCo/checkpoint_0199.pth.tar" #path to moco pretrained checkpoint
    size = 24 #adjust per dataset too
    num_classes = 6 #adjust per dataset

    # Custom additions for MoCo-specific configs
    moco_dim = 2048
    moco_k = 65536
    moco_m = 0.999
    moco_t = 0.07
    mlp = False
    aug_plus = True
    cos = False

# Usage
args = Args()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_acc1 = 0


def linclsmain():

    if args.seed is not None:
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        cudnn.deterministic = True
        warnings.warn(
            "You have chosen to seed training. "
            "This will turn on the CUDNN deterministic setting, "
            "which can slow down your training considerably! "
            "You may see unexpected behavior when restarting "
            "from checkpoints."
        )

    if args.gpu is not None:
        warnings.warn(
            "You have chosen a specific GPU. This will completely "
            "disable data parallelism."
        )

    if args.dist_url == "env://" and args.world_size == -1:
        args.world_size = int(os.environ["WORLD_SIZE"])

    args.distributed = args.world_size > 1 or args.multiprocessing_distributed

    ngpus_per_node = torch.cuda.device_count()

    main_worker(args.gpu, ngpus_per_node, args)


def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu

    # suppress printing if not master
    if args.multiprocessing_distributed and args.gpu != 0:

        def print_pass(*args):
            pass

        builtins.print = print_pass

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # create model
    print("=> creating model '{}'".format(args.arch))
    model = CustomImageClassifier(num_classes = args.num_classes, dim = args.moco_dim)
    model.to(device)

    # freeze all layers but the last fc
    for name, param in model.named_parameters():
        if name not in ["fc.weight", "fc.bias"]:
            param.requires_grad = False
    # init the fc layer
    model.fc.weight.data.normal_(mean=0.0, std=0.01)
    model.fc.bias.data.zero_()

    if args.pretrained:
      if os.path.isfile(args.pretrained):
        print("=> loading checkpoint '{}'".format(args.pretrained))
        checkpoint = torch.load(args.pretrained, map_location="cpu")
        state_dict = checkpoint["state_dict"]

        # Print keys to debug
        print("Keys in the loaded state_dict:", state_dict.keys())

        # Create a new state dictionary with adjusted keys
        new_state_dict = {}
        for k, v in state_dict.items():
            new_key = k.replace("module.encoder_q.", "")  # Adjust based on actual prefix and structure
            if "fc" not in new_key:  # Ignore fully connected layers if they should not be loaded
                new_state_dict[new_key] = v

        # Load the new state dictionary
        model.load_state_dict(new_state_dict, strict=False)
        print("=> loaded pre-trained model '{}'".format(args.pretrained))
      else:
        print("=> no checkpoint found at '{}'".format(args.pretrained))

    if args.distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if args.gpu is not None:
            model.to(device)
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            args.batch_size = int(args.batch_size / ngpus_per_node)
            args.workers = int((args.workers + ngpus_per_node - 1) / ngpus_per_node)
            model = torch.nn.parallel.DistributedDataParallel(
                model, device_ids=[args.gpu]
            )
        else:
            model.to(device)
            # DistributedDataParallel will divide and allocate batch_size to all
            # available GPUs if device_ids are not set
            model = torch.nn.parallel.DistributedDataParallel(model)
    elif args.gpu is not None:
        model = model.to(device)
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        if args.arch.startswith("alexnet") or args.arch.startswith("vgg"):
            model.features = torch.nn.DataParallel(model.features)
            model.to(device)
        else:
            model = torch.nn.DataParallel(model).to(device)

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().to(device)

    # optimize only the linear classifier
    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    assert len(parameters) == 2  # fc.weight, fc.bias
    optimizer = torch.optim.SGD(
        parameters, args.lr, momentum=args.momentum, weight_decay=args.weight_decay
    )

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            if args.gpu is None:
                checkpoint = torch.load(args.resume, map_location=device)
            else:
                checkpoint = torch.load(args.resume, map_location=device)
            args.start_epoch = checkpoint["epoch"]
            best_acc1 = checkpoint["best_acc1"]
            if args.gpu is not None:
                # best_acc1 may be from a checkpoint from a different GPU
                best_acc1 = best_acc1.to(args.gpu)
            model.load_state_dict(checkpoint["state_dict"])
            optimizer.load_state_dict(checkpoint["optimizer"])
            print(
                "=> loaded checkpoint '{}' (epoch {})".format(
                    args.resume, checkpoint["epoch"]
                )
            )
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    traindir = os.path.join(args.data, "train")
    valdir = os.path.join(args.data, "val")
    normalize = transforms.Normalize(
        mean=[0.485], std=[0.229]
    )

    augmentations = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.RandomResizedCrop(args.size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ]
        )


    train_dataset = LinClrDataset(args.data, mode='train', transform=augmentations)

    if args.distributed:
        train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
    else:
        train_sampler = None

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=(train_sampler is None),
        num_workers=args.workers,
        pin_memory=True,
        sampler=train_sampler,
    )

    val_dataset = LinClrDataset(args.data, mode='val', transform=augmentations)


    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=args.workers,
        pin_memory=True,
    )

    if args.evaluate:
        validate(val_loader, model, criterion, args)
        return

    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            train_sampler.set_epoch(epoch)
        adjust_learning_rate(optimizer, epoch, args)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, args)

        # evaluate on validation set
        acc1 = validate(val_loader, model, criterion, args)

        # remember best acc@1 and save checkpoint
        is_best = acc1 > best_acc1
        best_acc1 = max(acc1, best_acc1)

        if not args.multiprocessing_distributed or (
            args.multiprocessing_distributed and args.rank % ngpus_per_node == 0
        ):
            save_checkpoint(
                {
                    "epoch": epoch + 1,
                    "arch": args.arch,
                    "state_dict": model.state_dict(),
                    "best_acc1": best_acc1,
                    "optimizer": optimizer.state_dict(),
                },
                is_best,
            )
            if epoch == args.start_epoch:
                sanity_check(model.state_dict(), args.pretrained)


def train(train_loader, model, criterion, optimizer, epoch, args):
    batch_time = AverageMeter("Time", ":6.3f")
    data_time = AverageMeter("Data", ":6.3f")
    losses = AverageMeter("Loss", ":.4e")
    top1 = AverageMeter("Acc@1", ":6.2f")
    top5 = AverageMeter("Acc@5", ":6.2f")
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch),
    )

    """
    Switch to eval mode:
    Under the protocol of linear classification on frozen features/models,
    it is not legitimate to change any part of the pre-trained model.
    BatchNorm in train mode may revise running mean/std (even if it receives
    no gradient), which are part of the model parameters too.
    """
    model.eval()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if args.gpu is not None:
            images = images.to(device)
        target = target.to(device)

        if target.ndim > 1:
          target = target.squeeze()

        #print(target.max(), target.min())

        # compute output
        output = model(images)

        loss = criterion(output, target)

        # measure accuracy and record loss
        #acc1, acc5 = accuracy(output, target, topk=(1, 5))
        acc1 = accuracy(output, target)
        acc5 = acc1
        losses.update(loss.item(), images.size(0))
        top1.update(acc1, images.size(0))# top1.update(acc1[0], images.size(0))
        top5.update(acc5, images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            progress.display(i)


def validate(val_loader, model, criterion, args):
    batch_time = AverageMeter("Time", ":6.3f")
    losses = AverageMeter("Loss", ":.4e")
    top1 = AverageMeter("Acc@1", ":6.2f")
    top5 = AverageMeter("Acc@5", ":6.2f")
    precision = torchmetrics.Precision(num_classes=args.num_classes, average='weighted', task="multiclass").to(device)
    recall = torchmetrics.Recall(num_classes=args.num_classes, average='weighted', task="multiclass").to(device)
    f1 = torchmetrics.F1Score(num_classes=args.num_classes, average='weighted', task="multiclass").to(device)

    progress = ProgressMeter(
        len(val_loader), [batch_time, losses, top1, top5], prefix="Test: "
    )

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            #acc1, acc5 = accuracy(output, target, topk=(1, 5))
            acc1 = accuracy(output, target)
            acc5 = acc1
            losses.update(loss.item(), images.size(0))
            top1.update(acc1, images.size(0)) # top1.update(acc1[0], images.size(0))
            top5.update(acc5, images.size(0))

            #update metrics
            precision.update(output, target)
            recall.update(output, target)
            f1.update(output, target)

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(
            f" * Acc@1 {top1.avg:.4f} Acc@5 {top5.avg:.4f} Precision {precision.compute().item():.4f} Recall {recall.compute().item():.4f} F1 Score {f1.compute().item():.4f}"
        )

    return top1.avg


def save_checkpoint(state, is_best, filename="checkpoint.pth.tar"):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, "model_best.pth.tar")


def sanity_check(state_dict, pretrained_weights):
    """
    Linear classifier should not change any weights other than the linear layer.
    This sanity check asserts nothing wrong happens (e.g., BN stats updated).
    """
    print("=> loading '{}' for sanity check".format(pretrained_weights))
    checkpoint = torch.load(pretrained_weights, map_location="cpu")
    state_dict_pre = checkpoint["state_dict"]

    for k in list(state_dict.keys()):
        # only ignore fc layer
        if "fc.weight" in k or "fc.bias" in k:
            continue

        # name in pretrained model
        if k.startswith("module."):
            k_pre = "encoder_q.net." + k[len("module."):]  # Adjust based on your print output
        else:
            k_pre = "encoder_q.net." + k  # Assuming direct match after "encoder_q.net."

        # Check if the adjusted key exists in the pre-trained model state dict
        if k_pre in state_dict_pre:
            assert (state_dict[k].cpu() == state_dict_pre[k_pre]).all(), "{} is changed in linear classifier training.".format(k)
        else:
            print("Key not found in pre-trained model:", k_pre)

    print("=> sanity check passed.")


class AverageMeter:
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=":f"):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = "{name} {val" + self.fmt + "} ({avg" + self.fmt + "})"
        return fmtstr.format(**self.__dict__)


class ProgressMeter:
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print("\t".join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = "{:" + str(num_digits) + "d}"
        return "[" + fmt + "/" + fmt.format(num_batches) + "]"


def adjust_learning_rate(optimizer, epoch, args):
    """Decay the learning rate based on schedule"""
    lr = args.lr
    for milestone in args.schedule:
        lr *= 0.1 if epoch >= milestone else 1.0
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

'''
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
'''
def accuracy(output, target):
    pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
    correct = pred.eq(target.view_as(pred)).sum().item()
    return correct / output.shape[0]


In [8]:
linclsmain()

<ipython-input-7-dc0e6f767fb6>:101: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn(


Use GPU: 0 for training
=> creating model 'resnet50'
=> loading checkpoint '/content/drive/MyDrive/FYP_Nur_Time_Series_Representation_using_CL-main/MoCo/checkpoint_0199.pth.tar'
Keys in the loaded state_dict: odict_keys(['queue', 'queue_ptr', 'encoder_q.net.conv1.weight', 'encoder_q.net.bn1.weight', 'encoder_q.net.bn1.bias', 'encoder_q.net.bn1.running_mean', 'encoder_q.net.bn1.running_var', 'encoder_q.net.bn1.num_batches_tracked', 'encoder_q.net.layer1.0.conv1.weight', 'encoder_q.net.layer1.0.bn1.weight', 'encoder_q.net.layer1.0.bn1.bias', 'encoder_q.net.layer1.0.bn1.running_mean', 'encoder_q.net.layer1.0.bn1.running_var', 'encoder_q.net.layer1.0.bn1.num_batches_tracked', 'encoder_q.net.layer1.0.conv2.weight', 'encoder_q.net.layer1.0.bn2.weight', 'encoder_q.net.layer1.0.bn2.bias', 'encoder_q.net.layer1.0.bn2.running_mean', 'encoder_q.net.layer1.0.bn2.running_var', 'encoder_q.net.layer1.0.bn2.num_batches_tracked', 'encoder_q.net.layer1.0.conv3.weight', 'encoder_q.net.layer1.0.bn3.weight

In [9]:
from google.colab import runtime
runtime.unassign()